In [ ]:
import os

DEV_SET_FRAC = 0.001
OVERWRITE_SETS_WITH_DEV = True
os.environ["WANDB_PROJECT"]="lang-based-yappers/amazon_sentiment_analysis"
SEED = 1337

In [ ]:
import random

import numpy as np
import torch

torch.use_deterministic_algorithms(True)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print('Device:', device)

def set_seed():
    random.seed(SEED)
    np.random.seed(SEED)
    
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("amazon_polarity")
dataset

In [ ]:
train_ds, test_ds = dataset['train'], dataset['test']
train_df, test_df = train_ds.to_pandas(), test_ds.to_pandas()

In [ ]:
train_df_dev = train_df.sample(frac=DEV_SET_FRAC, random_state=1337)
test_df_dev = test_df.sample(frac=DEV_SET_FRAC, random_state=1337)

if OVERWRITE_SETS_WITH_DEV:
    train_df = train_df_dev
    test_df = test_df_dev
    
    train_ds = train_ds.select(range(len(train_df)))
    test_ds = test_ds.select(range(len(test_df)))

y_train, y_test = train_df['label'], test_df['label']
print(f"Training data shape: {train_df_dev.shape}, Testing data shape: {test_df_dev.shape}")

# Embedding

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def fit_tf_idf_vectorizer(train_df, test_df):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
    X_train_tfidf = vectorizer.fit_transform(train_df['content'])
    X_test_tfidf = vectorizer.transform(test_df['content'])
    return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = fit_tf_idf_vectorizer(train_df, test_df)

## BERT with Average Pooling

In [ ]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased", force_download=True)

model = BertModel.from_pretrained("google-bert/bert-base-uncased", force_download=True)
model.to(device)

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        return last_hidden_states

def get_sentence_embedding(text):
    with torch.no_grad():
        last_hidden_states = get_bert_embedding(text)
        return torch.mean(last_hidden_states, dim=1).squeeze()
    
def fit_bert_vectorizer(train_df, test_df):
    X_train_bert = torch.stack([get_sentence_embedding(text) for text in train_df['content']])
    X_test_bert = torch.stack([get_sentence_embedding(text) for text in test_df['content']])
    return X_train_bert, X_test_bert

X_train_bert, X_test_bert = fit_bert_vectorizer(train_df, test_df)

## KNeighbors Weak Labeling

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_bert, y_train)

In [ ]:
y_pred = knn.predict(X_test_bert)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(test_df['label'], y_pred))
print(f"Accuracy: {accuracy_score(test_df['label'], y_pred)}")